In [5]:
from bs4 import BeautifulSoup
import re
import requests
import fake_useragent
import json
import csv
import random
import time
import os
import pandas as pd

ua = fake_useragent.UserAgent( )

headers = {
    "Accept": "*/*",
    "User-Agent": ua.ie,
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
}

In [2]:
dic = {'А':'A', 'В':'B', 'Е':'E', 'К':'K', 'М':'M', 'Н':'H', 'О':'O', 'Р':'P', 
       'С':'C', 'Т':'T', 'У':'Y', 'Х':'X', 'х':'x'}

def Rustoeng(str):
    res=''
    for i in range(0,len(str)):
        if str[i] in dic:
            res =  res + dic[str[i]]
        else:
            res = res + str[i]
    return res

In [7]:
#Загрузка разделов датчиков ТЕКО
url = 'https://teko-com.ru/katalog/'
req = requests.get(url, headers=headers)
    
soup = BeautifulSoup(req.content, 'lxml')
all_sensors_href = soup.find('ul', class_="catalog-prod-ul").find_all('a')
all_catagories_dict = {}

for item in all_sensors_href:
    item_text = item.text
    item_href = 'https://teko-com.ru' + item.get('href')
    
    all_catagories_dict[item_text] = item_href
    
with open ('all_catagories_dict.json', "w") as file:
    json.dump(all_catagories_dict, file, indent = 4, ensure_ascii=False)
    
with open ('all_catagories_dict.json') as file:
    all_catagories = json.load(file)

print (all_catagories)

{'Индуктивные датчики': 'https://teko-com.ru/katalog/induktivnye-datchiki/', 'Оптические датчики': 'https://teko-com.ru/katalog/opticheskie-datchiki/', 'Емкостные датчики': 'https://teko-com.ru/katalog/emkostnye-datchiki/', 'Сенсорные кнопки': 'https://teko-com.ru/katalog/sensornye-knopki/', 'Магниточувствительные датчики': 'https://teko-com.ru/katalog/magnitochuvstvitelnye-datchiki/', 'Датчики уровня': 'https://teko-com.ru/katalog/datchiki-urovnja/', 'Ультразвуковые датчики': 'https://teko-com.ru/katalog/ultrazvukovie-datchiki/', 'Приборы и средства автоматизации': 'https://teko-com.ru/katalog/pribory/', 'Взрывозащищенное оборудование': 'https://teko-com.ru/katalog/vzryvobezopasnoe-oborudovanie/', 'Аксессуары к датчикам': 'https://teko-com.ru/katalog/aksessuary/', 'Конвейерная автоматика': 'https://teko-com.ru/katalog/konvejernaja-avtomatika/', 'Датчики относительной влажности и температуры воздуха': 'https://teko-com.ru/katalog/datchiki-otnositelnoj-vlazhnosti-i-temperatury/', 'Датчи

In [8]:
# Загрузка индуктивных датчиков с сайта ТЕКО
category_name = 'Индуктивные датчики'
category_href = 'https://teko-com.ru/katalog/induktivnye-datchiki/'

with open(f'data/Индуктивные датчики TEKO и LANBAO.csv', 'w', encoding = 'utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(('category_name', 'sensor_type', 'sensor_symbol', 'sensor_url', 'sensor_imgurl', 'sensor_price', 'sensor_in_stock'))
    
pages = 50
count = 0
for i in range(1,pages):
    
    url_page = category_href + '?limit=100&'+'page='+f'{i}'
    res = requests.get(url_page, headers=headers)
    soup = BeautifulSoup(res.content, 'lxml')
    sensor_list = soup.find_all('div', class_="product-thumb")
    if len(sensor_list) == 0:
        break
    for sensor in sensor_list:
        try:   
            sensor_bare_name = sensor.find('div', class_="tovarszg").text
        except:
            continue
        sensor_name = re.sub ( '\(.+\)', '', sensor_bare_name).strip()
        sensor_type = (re.sub("[a-zA-Z0-9-,]+", '', sensor_name).strip())
        sensor_type = ' '.join(filter(lambda x: len(x) > 4, sensor_type.split()))      #удалить короткие слова, фильтр русских букв в названии датчика попавших из обозначения
        sensor_symbol = Rustoeng(sensor_name.replace(sensor_type, '').strip())          #Транслитерация русских букв в обозначении
        sensor_url = sensor.find('a').get("href")
        sensor_imgurl = sensor.find('div', class_="image").find('img').get("src")
        sensor_price = sensor.find('span', class_="category-price-test").text.strip()
        sensor_in_stock = sensor.find('p', class_="pricers").find('span').text.strip()
        print (f'{sensor_type} - {sensor_symbol}')
        count += 1
        with open(f'data/Индуктивные датчики TEKO и LANBAO.csv', 'a', encoding = 'utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow((category_name, sensor_type, sensor_symbol, sensor_url, sensor_imgurl, sensor_price, sensor_in_stock))
    
    time.sleep(random.randrange(3,6))

Бесконтактный выключатель - ISB AF4B-31P-5-LZ-4,5
Бесконтактный выключатель - ISN ET8A-31N-15-LZ
Бесконтактный выключатель - ISB A4A8-31P-5F-LZT1-C-P
Бесконтактный выключатель - ISB AC8A-43P-10-LZS4
Бесконтактный выключатель - ISBt AC27B8-32P-4F-LTR9-C-P1-0,2
Бесконтактный выключатель - ISN F4A-32P-8-LZ
Бесконтактный выключатель - ISB A4B-31P-5-LZ
Бесконтактный выключатель - ISB WC29S8-31P-1,5-ZS4-2
Бесконтактный выключатель - ISB B1B-31P-1,5F-LP-C
Бесконтактный выключатель - ISN E73A-02G-15-L
Бесконтактный выключатель - ISB AF42A-11G-5-LZ-C
Бесконтактный выключатель - ISB AF9A-31P-12-LZ
Бесконтактный выключатель - ISN ET6A-43P-10-LZ
Бесконтактный выключатель - ISN F4A-31P-8-LZ
Бесконтактный выключатель - ISN ET8A-43N-15-LZ-C
Бесконтактный выключатель - ISN E4A-31P-8-LZ
Бесконтактный выключатель - ISB I27A-11-3,5-L-0,15
Бесконтактный выключатель - ISB AF41A-31P-5-LZ-C
Бесконтактный выключатель - ISB B2B-32P-4-L-D
Индуктивный преобразователь перемещения - ISAN E41A-31P-8-P
Бесконтактный

KeyboardInterrupt: 

In [41]:
def getimage(url, index, df):
    if df['sensor_imgurl'].isnull()[index]:
        return '/static/main/img/ind_TEKO/TEKO.jpeg'
    file_name = url.split('/')[-1]
    file_adr = '../main/static/main/img/ind_TEKO/' + file_name
    if not os.path.exists(file_adr):
        time.sleep(random.randrange(4,20))
        img_data = requests.get(url).content
        with open(file_adr, 'wb') as handler:
            handler.write(img_data)
    return '/static/main/img/ind_TEKO/' + file_name

In [46]:
with open(f'data/Индуктивные датчики TEKO и LANBAO.csv', encoding = 'utf-8') as f:
    df = pd.read_csv(f)
    
df.drop_duplicates(subset=['sensor_symbol'], keep='first', inplace=True)

df = df.assign(manufacturer = 'TEKO' ,
                       flush = None,
                       spec = None,
                       housing = None,
                       connection = None,
                       size = None,
                       material = None,
                       IP = None,
                       output_structure = None,
                       voltage = None,
                       output_type = None,
                       adjust = None,
                       distance = None,
                       indicator = None,
                       temperature = None)


df['sensor_symbol'] = df.sensor_symbol.str.replace('IV1N-I7P5-43P-R50-LZ', 'IV1N I7P5-43P-R50-LZ')
df['sensor_symbol'] = df.sensor_symbol.str.replace('BK WF63-31-P-3-400-ЗB-1-HT', 'ISB WF63A8-31P-3-Z-1C')
df['sensor_symbol'] = df.sensor_symbol.str.replace('AF7A-10S-N-C-25', 'ISB AF7A-10S-N-C-25')

for index, row in df.iterrows():
    row['sensor_symbol'] = Rustoeng(row['sensor_symbol'])


# Тип датчика
df.loc [df.sensor_symbol.str.contains('IV'), 'sensor_type'] = 'Датчик скорости'
df.loc [df.sensor_symbol.str.contains('Д[КK]М?[CС]'), 'sensor_type'] = 'Датчик скорости'
df.loc [df.sensor_symbol.str.contains('^ISA'), 'sensor_type'] = 'Индуктивный преобразователь перемещения'
df.loc [df.sensor_symbol.str.contains('^IS[BN]'), 'sensor_type'] = 'Индуктивный датчик'

df.loc [df.sensor_symbol.str.contains('(\.|^)(\d\d\d\d.*)$'), 'sensor_symbol'] = df.sensor_symbol.str.replace(' ', '')
df.loc [df.sensor_symbol.str.contains('(\.|^)(\d\d\d\d.*)$'), 'sensor_symbol'] = df.sensor_symbol.str.replace('[a-zA-Zа-яА-Я]', '')
df.loc [df.sensor_symbol.str.contains('(\.|^)(\d\d\d\d.*)$'), 'sensor_symbol'] = 'ВТИЮ.' + df.sensor_symbol.str.replace('^\.', '')
df.loc [df.sensor_symbol.str.contains('ВТИЮ'), 'sensor_type'] = 'Индуктивный датчик ВТИЮ.хххх'

df.loc [df.sensor_symbol.str.contains('ВТИЮ.78-32-40----1078-32.1349'), 'sensor_symbol'] = 'ISN IF7P8-32P-40-LZ-C-V-10'
df.loc [df.sensor_symbol.str.contains('ВТИЮ.78-32-40---78-32.1349'), 'sensor_symbol'] = 'ISN IF7P8-32P-40-LZ-C-V'
# Производитель
df.loc [df.sensor_symbol.str.contains('^I[SV]'), 'manufacturer'] = 'TEKO'
df.loc [df.sensor_symbol.str.contains('Д[КK]М?[CС]'), 'manufacturer'] = 'TEKO'
df.loc [df.sensor_symbol.str.contains('^[LF]'), 'manufacturer'] = 'LANBAO'
df.loc [df.sensor_symbol.str.contains('ВТИЮ'), 'manufacturer'] = 'TEKO'
df = df.loc[df['manufacturer'] != 'LANBAO']                                         #Убрать датчики LANBAO
# Цена
df['sensor_price'] = df['sensor_price'].str.replace(' ','')
df['sensor_price'] = df.sensor_price.str.extract(r'([0-9]+)')
df['sensor_price'] = df['sensor_price'].str.replace(',','.')
# Cпособ установки
df.loc [df.sensor_symbol.str.contains('\w{2,3}N[mt\s]'), 'flush'] = 'Не встраиваемый'
df.loc [df.sensor_symbol.str.contains('\w{2,3}B[mt\s]'), 'flush'] = 'Встраиваемый'
df.loc [df.sensor_symbol.str.contains('^Д[КK][СC]'), 'flush'] = 'Встраиваемый'

# Исполнение
df.loc [df.sensor_symbol.str.contains('[BN]\s'), 'spec'] = 'Общее применение'
df.loc [df.sensor_symbol.str.contains('^Д[КK][СC]'), 'spec'] = 'Общее применение'
df.loc [df.sensor_symbol.str.contains('[BN]m.'), 'spec'] = 'Морское исполнение'
df.loc [df.sensor_symbol.str.contains('[BN]t.'), 'spec'] = 'Транспортное исполнение'
df.loc [df.sensor_symbol.str.contains('ВТИЮ'), 'spec'] = 'Специальный'
df.loc [df.sensor_symbol.str.contains('\sWB?[FCTG]?\d'), 'spec'] = 'В среде высокого давления'
df.loc [df.sensor_symbol.str.contains('-N-|-N$'), 'spec'] = 'Во взрывоопасных зонах (NAMUR)'
df.loc [df.sensor_symbol.str.contains('\w-V'), 'spec'] = 'В условиях повышенной вибрации '

# Конструктивное исполнение корпуса
df.loc [df.sensor_symbol.str.contains('\s[ABEF]S?P?[FCTG]?\d'), 'housing'] = 'Цилиндрический с резьбой'
df.loc [df.sensor_symbol.str.contains('\s[CDGH]S?[FCTG]?\d'), 'housing'] = 'Цилиндрический гладкий'
df.loc [df.sensor_symbol.str.contains('\sI[FCTG]?\d'), 'housing'] = 'Прямоугольный'
df.loc [df.sensor_symbol.str.contains('BRPU4P'), 'housing'] = 'Прямоугольный'
df.loc [df.sensor_symbol.str.contains('\sIm'), 'housing'] = 'Прямоугольный'
df.loc [df.sensor_symbol.str.contains('\sK[FCTG]?\d'), 'housing'] = 'Cпец. назначения для кранов, клапанов, поворотных задвижек'
df.loc [df.sensor_symbol.str.contains('\sL[FCTG]?\d'), 'housing'] = 'Плоский'
df.loc [df.sensor_symbol.str.contains('\sM[FCTG]?\d'), 'housing'] = 'Уголковый'
df.loc [df.sensor_symbol.str.contains('\sN[FCTG]?\d'), 'housing'] = 'Щелевой'
df.loc [df.sensor_symbol.str.contains('\sR[FCTG]?\d'), 'housing'] = 'Кольцевой'
df.loc [df.sensor_symbol.str.contains('\sT[FCTG]?\d'), 'housing'] = 'Для многопозиционных блоков'
df.loc [df.sensor_symbol.str.contains('\sZ[FCTG]?\d'), 'housing'] = 'Cпециального конструктивного исполнения'
df.loc [df.sensor_symbol.str.contains('\sWB?[FCTG]?\d'), 'housing'] = 'Цилиндрический с резьбой'
df.loc [df.sensor_symbol.str.contains('^Д[КK][СC]'), 'housing'] = 'Цилиндрический с резьбой'

# Cпособ подключения
df['connection']  = 'Кабель'
df.loc [df.sensor_symbol.str.contains('\wF\d'), 'connection'] = 'Кабель (фиттинг)'
df.loc [df.sensor_symbol.str.contains('\wC\d'), 'connection'] = 'Разъём'
df.loc [df.sensor_symbol.str.contains('\wT\d'), 'connection'] = 'Клеммы'
df.loc [df.sensor_symbol.str.contains('\wG\d'), 'connection'] = 'Кабель'
# Размер корпуса
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[ABEF]S?[FCTG]?0'), 'size'] = '∅5'     #0 (∅4; М5х0,5; ∅6,5)
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[DGH]S?[FCTG]?0'), 'size'] = '∅6,5'     #0
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[C]S?[FCTG]?0'), 'size'] = '∅4'     #0

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[A-H]S?[FCTG]?1(?!0)'), 'size'] = '∅8'     #1  (∅8; М8х0,5; М8х1)
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[A-H]S?[FCTG]?2'), 'size'] = '∅12'     #2

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[ABEF]S?[FCTG]?3'), 'size'] = '∅16'     #3 *(∅14; М14х1; М16х1)
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[CDGH]S?[FCTG]?3'), 'size'] = '∅14'     #3

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[ABEF]S?[FCTG]?4'), 'size'] = '∅18'     #4 (∅17; М18х1)
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[CDGH]S?[FCTG]?4'), 'size'] = '∅17'     #4 ()

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[ABEF]S?[FCTG]?5'), 'size'] = '∅22'     #5 (∅20; М22х1,5 (укороченные); ∅55 (невстраиваемые))
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[CDG]S?[FCTG]?5'), 'size'] = '∅20'     #5 (∅20; М22х1,5 (укороченные); ∅55 (невстраиваемые))
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[H]S?[FCTG]?5'), 'size'] = '∅55'     #5 (∅20; М22х1,5 (укороченные); ∅55 (невстраиваемые))

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[ABEF]S?[FCTG]?6'), 'size'] = '∅20'     #6 (М22х1,5; М24х1,5)
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[H]S?[FCTG]?6'), 'size'] = '∅75'     #6 (М22х1,5; М24х1,5)

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[AE]S?[FCTG]?7'), 'size'] = '∅27'     #7(М27х1,5; М30х1,5 (укороченные))
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[BF]S?[FCTG]?7'), 'size'] = '∅30'     #7(М27х1,5; М30х1,5 (укороченные))

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[AE]S?[FCTG]?8'), 'size'] = '∅30'     #8 (М30х1,5; М36х1,5 (укороченные))
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[BF]S?[FCTG]?8'), 'size'] = '∅36'     #8 (М30х1,5; М36х1,5 (укороченные))

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[ABEF]S?[FCTG]?9'), 'size'] = '∅36'     #9  (М36х1,5)
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[A-H]S?[FCTG]?10'), 'size'] = '∅55'     #10  (∅55 (встраиваемые)

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[W]B?S?[FCTG]?2'), 'size'] = '∅12'     #W2
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[W]B?S?[FCTG]?3'), 'size'] = '∅16'     #W3
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[W]B?S?[FCTG]?4'), 'size'] = '∅18'     #W4
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[W]B?S?[FCTG]?6'), 'size'] = '∅22'     #W6
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[W]B?S?[FCTG]?7'), 'size'] = '∅27'     #W7

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?1'), 'size'] = '20х28х66'       #I1
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?2'), 'size'] = '26х40х12'       #I2
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?4'), 'size'] = '25х50х10'       #I4
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?5'), 'size'] = '42х48х22'       #I5
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?6'), 'size'] = '60,5x74x22'       #I6
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?[-\s][I]S?[FCTG]?7'), 'size'] = '80х80х40'       #I7
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?8(?!\d)'), 'size'] = '60x60x40'       #I8
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?81'), 'size'] = '60х60х50'       #I81
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?82'), 'size'] = '55х47х22'       #I82
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?16'), 'size'] = '170х170х60'       #I16
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?121'), 'size'] = '40х40х120'       #I121
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?123'), 'size'] = '20х30х10'       #I123
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[I]S?[FCTG]?131'), 'size'] = '40x40x63'       #I131
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[Im]S?[FCTG]?'), 'size'] = '84х64х43'       #Im

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[L]S?[FCTG]?2'), 'size'] = '75x78x11,5'       #L2
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[L]S?[FCTG]?3'), 'size'] = '64x65x11,5'       #L3

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[M]S?[FCTG]?3'), 'size'] = '75x81x27,5'       #M3
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[M]S?[FCTG]?4'), 'size'] = '65x67,5x27,5'       #M4
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[M]S?[FCTG]?5'), 'size'] = '65x77x30'       #M5
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[M]S?[FCTG]?6'), 'size'] = '48x70x35'       #M6

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[N]S?[FCTG]?2'), 'size'] = '25x28x18'       #N2
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[N]S?[FCTG]?3'), 'size'] = '45x40x40'       #N3

df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[R]S?[FCTG]?1'), 'size'] = '40х30х70'       #R1
# Материал корпуса
df.loc [df.sensor_symbol.str.contains('\w[\wm]P[\d-]'), 'material'] = 'Пластик'
df.loc [df.sensor_symbol.str.contains('\w\dB[\d-]'), 'material'] = 'Латунь'
df.loc [df.sensor_symbol.str.contains('\w\dA[\d-]'), 'material'] = 'Алюминий'
df.loc [df.sensor_symbol.str.contains('\w\d[SF][\d-]'), 'material'] = 'Нержавеющая сталь'
# Cтепень защиты (нет - IP67; 5 - IP65; 8 - IP68;)
df['IP'] = 'IP 67'
df.loc [df.sensor_symbol.str.contains('[A-Z]8-\d'), 'IP'] = 'IP 68'
df.loc [df.sensor_symbol.str.contains('[A-Z]5-\d'), 'IP'] = 'IP 65'
# Cтруктура выходного сигнала
df.loc [df.sensor_symbol.str.contains('^I[VS]\d?\d?[NB][\w\s]+-[0129]\d\d?[GS]?-'), 'output_structure'] = 'Двухпроводный датчик'
df.loc [df.sensor_symbol.str.contains('^I[VS]\d?\d?[NB][\w\s]+-[58]\d\d?[GS]?-'), 'output_structure'] = 'Реле'
df.loc [df.sensor_symbol.str.contains('^I[VS]\d?\d?[NB][\w\s]+-\d{1,3}N[GS]?-'), 'output_structure'] = 'NPN'
df.loc [df.sensor_symbol.str.contains('^I[VS]\d?\d?[NB]-?[\w\s]+-\d{1,3}P[GS]?-'), 'output_structure'] = 'PNP'
df.loc [df.sensor_symbol.str.contains('-N-|-N$'), 'output_structure'] = 'NAMUR'
df.loc [df.sensor_symbol.str.contains('^I[VS]\d?\d?[NB][\w\s]+-\d[78]\d?[NP]?[GS]?-'), 'output_structure'] = 'Оптрон'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d[15]\d?[NP]?[GS]?-'), 'output_structure'] = 'PNP'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d2\d?[NP]?[GS]?-'), 'output_structure'] = 'PNP'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d3\d?[NP]?[GS]?-'), 'output_structure'] = 'PNP'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d4\d?[NP]?[GS]?-'), 'output_structure'] = 'PNP'
# Напряжение
df.loc [df.sensor_symbol.str.contains('-[0678]\d'), 'voltage'] = '90-250 [AC]'
df.loc [df.sensor_symbol.str.contains('-1\d'), 'voltage'] = '20-320 [DC] / 20-250 [AC]'
df.loc [df.sensor_symbol.str.contains('-[2-5]\d'), 'voltage'] = '10-30 [DC]'
df.loc [df.sensor_symbol.str.contains('-\d\d1'), 'voltage'] = '3-15 [DC]'
df.loc [df.sensor_symbol.str.contains('-\d\d2'), 'voltage'] = '10-65 [DC]'
df.loc [df.sensor_symbol.str.contains('-\d\d3'), 'voltage'] = '15-150 [DC]'
df.loc [df.sensor_symbol.str.contains('-\d\d4'), 'voltage'] = '15-110 [DC]'
df.loc [df.sensor_symbol.str.contains('-\d\d5'), 'voltage'] = '77-150 [DC]'
df.loc [df.sensor_symbol.str.contains('-\d\d6'), 'voltage'] = '40-250 [AC]'
df.loc [df.sensor_symbol.str.contains('-\d\d9'), 'voltage'] = '320-420 [AC]'
df.loc [df.sensor_symbol.str.contains('-N-|-N$'), 'voltage'] = '7,7-9,0 [DC] (NAMUR)'
# Тип контакта
df.loc [df.sensor_symbol.str.contains('[A-Zm]\d{0,3}[ASFBP][58]?-\d[258]\d?[NP]?[GS]?-'), 'output_type'] = 'NC'
df.loc [df.sensor_symbol.str.contains('[A-Zm]\d{0,3}[ASFBP]F?L?[58]?-\d[147]\d?[NP]?[GS]?-'), 'output_type'] = 'NO'
df.loc [df.sensor_symbol.str.contains('-N-|-N$'), 'output_type'] = 'NC'
df.loc [df.sensor_symbol.str.contains('[A-Zm]\d{0,3}[ASFBP][58]?-\d[36]\d?[NP]?[GS]?-'), 'output_type'] = 'NO и NC'
df.loc [df.sensor_symbol.str.contains('[A-Zm]\d{0,3}[ASFBP][58]?-\d[9]\d?[NP]?[GS]?-'), 'output_type'] = 'NO или NC'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d[15]\d?[NP]?[GS]?-'), 'output_type'] = 'Напряжение'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d2\d?[NP]?[GS]?-'), 'output_type'] = 'Ток'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d3\d?[NP]?[GS]?-'), 'output_type'] = 'Напряжение и ток'
df.loc [df.sensor_symbol.str.contains('^I[VS]A\d?[NB][\w\s]+-\d4\d?[NP]?[GS]?-'), 'output_type'] = 'Напряжение и ток'
# Регулировка
df['adjust'] = 'Нет'
df.loc [df.sensor_symbol.str.contains('\w-R\d'), 'adjust'] = 'Есть'
# Номинальное расстояние
df['distance'] = df.sensor_symbol.str.extract(r'I[\w\s]+-?[\w\s]+-R?(\d\d?,?\.?\d?)[A-M]?[-]?')
df.loc [df.sensor_symbol.str.contains('BRPU4P'), 'distance'] = '10'
df['distance'] = df['distance'].str.replace(',','.').astype(float)
# Индикация
df['indicator'] = 'Нет'
df.loc [df.sensor_symbol.str.contains('\w-L[-ZEPT]'), 'indicator'] = 'Есть'
# Температурный диапазон
df['temperature'] = '-15...+65'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(C-|C$)'), 'temperature'] = '-45...+65'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(C1-|C1$)'), 'temperature'] = '-45...+90'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(T-|T$)'), 'temperature'] = '-25...+75'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(H-|H$)'), 'temperature'] = '-15...+105'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(CH-|CH$)'), 'temperature'] = '-40...+105'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(S-|S$)'), 'temperature'] = '-5...+120'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(K-|K$)'), 'temperature'] = '0...+150'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(D-|D$)'), 'temperature'] = '-60...+65'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(C2-|C2$)'), 'temperature'] = '-60...+90'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(Q-|Q$)'), 'temperature'] = '-15...+105'
df.loc [df.sensor_symbol.str.contains('.*-.*-.*-.*(G-|G$)'), 'temperature'] = '-5...+120'

# Срок отгрузки
df.loc [df.sensor_in_stock.str.contains('Пожалуйста'), 'sensor_in_stock'] = 'По запросу'

for index, row in df.iterrows():
    df.loc[index, 'sensor_imgurl'] = getimage(row['sensor_imgurl'], index, df)

# Почистить поля для датчиков ВТИЮ, так как None
df.loc [df.sensor_symbol.str.contains('ВТИЮ'), 
        ['housing', 'flush','material','voltage','adjust','indicator','distance',
         'temperature','size','output_type','output_structure','IP', 'connection']] = None


# Костыли
df['sensor_symbol'] = df.sensor_symbol.str.replace('Бесконтактный  выключатель ', '')
df['sensor_symbol'] = df.sensor_symbol.str.replace('Беcконтактный выключатель ', '')
df.loc [df.sensor_type.str.contains('^.+выключ.*'), 'sensor_type'] = 'Индуктивный датчик'
df.loc [df.sensor_symbol.str.contains('^I[VS]\w?\d?[NB][tm]?E?x?\s[A-H]S?P?[FCTG]?1(?!0)'), 'size'] = '∅8мм'     #1  (∅8; М8х0,5; М8х1)
df.loc [df.sensor_symbol.str.contains('^I[VS]\d?\d?[NB][\w\s]+-\d{1,3}N[GS]?-'), 'output_structure'] = 'NPN'




C:\Users\egors\AppData\Local\Temp\ipykernel_76316\1211203713.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc [df.sensor_symbol.str.contains('(\.|^)(\d\d\d\d.*)$'), 'sensor_symbol'] = df.sensor_symbol.str.replace(' ', '')
C:\Users\egors\AppData\Local\Temp\ipykernel_76316\1211203713.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc [df.sensor_symbol.str.contains('(\.|^)(\d\d\d\d.*)$'), 'sensor_symbol'] = df.sensor_symbol.str.replace('[a-zA-Zа-яА-Я]', '')
C:\Users\egors\AppData\Local\Temp\ipykernel_76316\1211203713.py:38: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc [df.sensor_symbol.str.contains('(\.|^)(\d\d\d\d.*)$'), 'sensor_symbol'] = df.sensor_symbol.str.replace('[a-zA-Zа-яА-Я]', '')
C:\Users\egors\AppData\Local\Temp\ipykernel_763

In [43]:
df = df.reindex(columns=['category_name', 'sensor_type', 'sensor_symbol', 'sensor_url', 'sensor_imgurl',
                         'sensor_price', 'sensor_in_stock', 'manufacturer', 'flush', 'spec', 'housing',
                         'connection', 'size', 'material', 'IP','output_structure','voltage',
                         'output_type','adjust','distance','indicator','temperature'])

new_columns = {'category_name': 'category',
               'sensor_type': 'type',
               'sensor_symbol': 'name',
               'sensor_url': 'url',
               'sensor_imgurl': 'url_img',
               'sensor_price': 'price',
               'sensor_in_stock': 'stock',
               }

df = df.rename(columns=new_columns)

df.to_csv('data/Индуктивные датчики ТЕКО dataframe.csv', index=False)

In [31]:
    
#         'category_name', 'sensor_type', 'sensor_symbol', 'sensor_url', 'sensor_imgurl', 
#         'sensor_price', 'sensor_in_stock', 'manufacturer', 'flush', 'spec', 'housing', 'connection', 
#         'size','material','IP','output_structure','voltage','output_type','adjust','distance','indicator','temperature'

# df['temperature'].unique()



array(['-15...+65', '-45...+65', '-60...+65', '-15...+105', '-60...+90',
       '0...+150', '-5...+120', None, '-40...+105', '-25...+75',
       '-45...+90'], dtype=object)

In [45]:
import sqlite3
import pandas as pd

sql_delete_query = 'DELETE FROM main_inductive'
conn  = sqlite3.connect('dbtest.sqlite3')
cursor = conn.cursor()
cursor.execute(sql_delete_query)
conn.commit()
cursor.close()

with open(f'data/Индуктивные датчики ТЕКО dataframe.csv', encoding = 'utf-8') as f:
    df = pd.read_csv(f)
conn  = sqlite3.connect('dbtest.sqlite3')
df.to_sql(name='main_inductive', con=conn, if_exists='append', index=False)
conn.close()

with open(f'data/Индуктивные датчики SKB Ind dataframe.csv', encoding = 'utf-8') as f:
    df = pd.read_csv(f)
conn  = sqlite3.connect('dbtest.sqlite3')
df.to_sql(name='main_inductive', con=conn, if_exists='append', index=False)
conn.close()
